# Summarizing Text 

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
from nltk.corpus import stopwords
import time
from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors
print('TensorFlow Version: {}'.format(tf.__version__))

TensorFlow Version: 1.15.0


In [2]:
with tf.Session() as sess:
  devices = sess.list_devices()
print (devices)

[_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456, 862031821031415827), _DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 14247736103356357571), _DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:0, GPU, 3542614016, 6682711524476484399), _DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 17626999181137717918)]


In [2]:
# ! pip install pandas
! pip install pandas


  Using cached pandas-1.0.3-cp37-cp37m-manylinux1_x86_64.whl (10.0 MB)
  Using cached pytz-2019.3-py2.py3-none-any.whl (509 kB)


## Insepcting the Data

In [7]:
import pandas as pd 
reviews = pd.read_csv("tr_physics.csv") 
reviews.head()

,id,Summary,categories,Text,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,704.1291,projective hilbert space structures at excepti...,math-ph cond-mat.other math.mp quant-ph,a non-hermitian complex symmetric 2x2 matrix t...,NaN,NaN,NaN,NaN
1,705.1265,a noncommutative bohnenblust-spitzer identity ...,math.co hep-th math-ph math.mp math.ra,the bogoliubov recursion is a particular proce...,NaN,NaN,NaN,NaN
2,705.4019,features of ion acceleration by circularly pol...,physics.plasm-ph,the characteristics of a mev ion source driven...,NaN,NaN,NaN,NaN
3,706.0838,periodicity of ~155 days in solar electron flu...,astro-ph,in this paper we have investigated the occurre...,NaN,NaN,NaN,NaN
4,706.1633,large attractive depletion interactions in sof...,cond-mat.soft cond-mat.stat-mech,we consider binary mixtures of soft repulsive ...,NaN,NaN,NaN,NaN


In [8]:
# Inspecting some of the reviews
for i in range(5):
    print("Review #",i+1)
    print(reviews.Summary[i])
    print(reviews.Text[i])
    print()

Review # 1
projective hilbert space structures at exceptional points
a non-hermitian complex symmetric 2x2 matrix toy model is used to study projective hilbert space structures in the vicinity of exceptional points (eps). the bi-orthogonal eigenvectors of a diagonalizable matrix are puiseux-expanded in terms of the root vectors at the ep. it is shown that the apparent contradiction between the two incompatible normalization conditions with finite and singular behavior in the ep-limit can be resolved by projectively extending the original hilbert space. the complementary normalization conditions correspond then to two different affine charts of this enlarged projective hilbert space. geometric phase and phase jump behavior are analyzed and the usefulness of the phase rigidity as measure for the distance to ep configurations is demonstrated. finally, ep-related aspects of pt-symmetrically extended quantum mechanics are discussed and a conjecture concerning the quantum brachistochrone pro

## Preparing the Data

In [9]:
# A list of contractions from http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [10]:
def clean_text(text, remove_stopwords = True):
    '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''
    
    # Convert words to lower case
    text = text.lower()
    
    # Replace contractions with their longer forms 
    if True:
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    # Format words and remove unwanted characters
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    
    # Optionally, remove stop words
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)

    return text

We will remove the stopwords from the texts because they do not provide much use for training our model. However, we will keep them for our summaries so that they sound more like natural phrases. 

In [11]:
# Clean the summaries and texts
clean_summaries = []
for summary in reviews.Summary:
    clean_summaries.append(clean_text(summary, remove_stopwords=False))
print("Summaries are complete.")

clean_texts = []
for text in reviews.Text:
    clean_texts.append(clean_text(text))
print("Texts are complete.")

Summaries are complete.
Texts are complete.


In [12]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/shourya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
# Inspect the cleaned summaries and texts to ensure they have been cleaned well
for i in range(5):
    print("Clean Review #",i+1)
    print(clean_summaries[i])
    print(clean_texts[i])
    print()

Clean Review # 1
projective hilbert space structures at exceptional points
non hermitian complex symmetric 2x2 matrix toy model used study projective hilbert space structures vicinity exceptional points eps bi orthogonal eigenvectors diagonalizable matrix puiseux expanded terms root vectors ep shown apparent contradiction two incompatible normalization conditions finite singular behavior ep limit resolved projectively extending original hilbert space complementary normalization conditions correspond two different affine charts enlarged projective hilbert space geometric phase phase jump behavior analyzed usefulness phase rigidity measure distance ep configurations demonstrated finally ep related aspects pt symmetrically extended quantum mechanics discussed conjecture concerning quantum brachistochrone problem formulated

Clean Review # 2
a noncommutative bohnenblust spitzer identity for rota baxter algebras solves bogoliubov s recursion
bogoliubov recursion particular procedure appeari

In [14]:
def count_words(count_dict, text):
    '''Count the number of occurrences of each word in a set of text'''
    for sentence in text:
        for word in sentence.split():
            if word not in count_dict:
                count_dict[word] = 1
            else:
                count_dict[word] += 1

In [15]:
# Find the number of times each word was used and the size of the vocabulary
word_counts = {}

count_words(word_counts, clean_summaries)
count_words(word_counts, clean_texts)
            
print("Size of Vocabulary:", len(word_counts))

Size of Vocabulary: 109592


In [16]:
# Load Conceptnet Numberbatch's (CN) embeddings, similar to GloVe, but probably better 
# (https://github.com/commonsense/conceptnet-numberbatch)
embeddings_index = {}
with open('numberbatch-en-17.02.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding

print('Word embeddings:', len(embeddings_index))

Word embeddings: 484557


In [18]:
# Find the number of words that are missing from CN, and are used more than our threshold.
missing_words = 0
threshold = 20

for word, count in word_counts.items():
    if count > threshold:
        if word not in embeddings_index:
            missing_words += 1
            
missing_ratio = round(missing_words/len(word_counts),4)*100
            
print("Number of words missing from CN:", missing_words)
print("Percent of words that are missing from vocabulary: {}%".format(missing_ratio))

Number of words missing from CN: 2311
Percent of words that are missing from vocabulary: 2.11%


I use a threshold of 20, so that words not in CN can be added to our word_embedding_matrix, but they need to be common enough in the reviews so that the model can understand their meaning.

In [19]:
# Limit the vocab that we will use to words that appear ≥ threshold or are in GloVe

#dictionary to convert words to integers
vocab_to_int = {} 

value = 0
for word, count in word_counts.items():
    if count >= threshold or word in embeddings_index:
        vocab_to_int[word] = value
        value += 1

# Special tokens that will be added to our vocab
codes = ["<UNK>","<PAD>","<EOS>","<GO>"]   

# Add codes to vocab
for code in codes:
    vocab_to_int[code] = len(vocab_to_int)

# Dictionary to convert integers to words
int_to_vocab = {}
for word, value in vocab_to_int.items():
    int_to_vocab[value] = word

usage_ratio = round(len(vocab_to_int) / len(word_counts),4)*100

print("Total number of unique words:", len(word_counts))
print("Number of words we will use:", len(vocab_to_int))
print("Percent of words we will use: {}%".format(usage_ratio))

Total number of unique words: 109592
Number of words we will use: 45714
Percent of words we will use: 41.71%


In [20]:
# Need to use 300 for embedding dimensions to match CN's vectors.
embedding_dim = 300
nb_words = len(vocab_to_int)

# Create matrix with default values of zero
word_embedding_matrix = np.zeros((nb_words, embedding_dim), dtype=np.float32)
for word, i in vocab_to_int.items():
    if word in embeddings_index:
        word_embedding_matrix[i] = embeddings_index[word]
    else:
        # If word not in CN, create a random embedding for it
        new_embedding = np.array(np.random.uniform(-1.0, 1.0, embedding_dim))
        embeddings_index[word] = new_embedding
        word_embedding_matrix[i] = new_embedding

# Check if value matches len(vocab_to_int)
print(len(word_embedding_matrix))

45714


In [21]:
def convert_to_ints(text, word_count, unk_count, eos=False):
    '''Convert words in text to an integer.
       If word is not in vocab_to_int, use UNK's integer.
       Total the number of words and UNKs.
       Add EOS token to the end of texts'''
    ints = []
    for sentence in text:
        sentence_ints = []
        for word in sentence.split():
            word_count += 1
            if word in vocab_to_int:
                sentence_ints.append(vocab_to_int[word])
            else:
                sentence_ints.append(vocab_to_int["<UNK>"])
                unk_count += 1
        if eos:
            sentence_ints.append(vocab_to_int["<EOS>"])
        ints.append(sentence_ints)
    return ints, word_count, unk_count

In [22]:
# Apply convert_to_ints to clean_summaries and clean_texts
word_count = 0
unk_count = 0

int_summaries, word_count, unk_count = convert_to_ints(clean_summaries, word_count, unk_count)
int_texts, word_count, unk_count = convert_to_ints(clean_texts, word_count, unk_count, eos=True)

unk_percent = round(unk_count/word_count,4)*100

print("Total number of words in headlines:", word_count)
print("Total number of UNKs in headlines:", unk_count)
print("Percent of words that are UNK: {}%".format(unk_percent))

Total number of words in headlines: 6355804
Total number of UNKs in headlines: 153640
Percent of words that are UNK: 2.42%


In [23]:
def create_lengths(text):
    '''Create a data frame of the sentence lengths from a text'''
    lengths = []
    for sentence in text:
        lengths.append(len(sentence))
    return pd.DataFrame(lengths, columns=['counts'])

In [24]:
lengths_summaries = create_lengths(int_summaries)
lengths_texts = create_lengths(int_texts)

print("Summaries:")
print(lengths_summaries.describe())
print()
print("Texts:")
print(lengths_texts.describe())

Summaries:
             counts
count  55521.000000
mean      11.427514
std        4.363274
min        1.000000
25%        8.000000
50%       11.000000
75%       14.000000
max       50.000000

Texts:
             counts
count  55521.000000
mean     104.048162
std       42.061018
min        2.000000
25%       73.000000
50%       99.000000
75%      132.000000
max      333.000000


In [25]:
# Inspect the length of texts
print(np.percentile(lengths_texts.counts, 90))
print(np.percentile(lengths_texts.counts, 95))
print(np.percentile(lengths_texts.counts, 99))

165.0
180.0
203.0


In [26]:
# Inspect the length of summaries
print(np.percentile(lengths_summaries.counts, 90))
print(np.percentile(lengths_summaries.counts, 95))
print(np.percentile(lengths_summaries.counts, 99))

17.0
19.0
24.0


In [27]:
def unk_counter(sentence):
    '''Counts the number of time UNK appears in a sentence.'''
    unk_count = 0
    for word in sentence:
        if word == vocab_to_int["<UNK>"]:
            unk_count += 1
    return unk_count

In [28]:
# Sort the summaries and texts by the length of the texts, shortest to longest
# Limit the length of summaries and texts based on the min and max ranges.
# Remove reviews that include too many UNKs

sorted_summaries = []
sorted_texts = []
max_text_length = 84
max_summary_length = 13
min_length = 2
unk_text_limit = 1
unk_summary_limit = 0

for length in range(min(lengths_texts.counts), max_text_length): 
    for count, words in enumerate(int_summaries):
        if (len(int_summaries[count]) >= min_length and
            len(int_summaries[count]) <= max_summary_length and
            len(int_texts[count]) >= min_length and
            unk_counter(int_summaries[count]) <= unk_summary_limit and
            unk_counter(int_texts[count]) <= unk_text_limit and
            length == len(int_texts[count])
           ):
            sorted_summaries.append(int_summaries[count])
            sorted_texts.append(int_texts[count])
        
# Compare lengths to ensure they match
print(len(sorted_summaries))
print(len(sorted_texts))

11029
11029


## Building the Model

In [29]:
def model_inputs():
    '''Create palceholders for inputs to the model'''
    
    input_data = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    lr = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    summary_length = tf.placeholder(tf.int32, (None,), name='summary_length')
    max_summary_length = tf.reduce_max(summary_length, name='max_dec_len')
    text_length = tf.placeholder(tf.int32, (None,), name='text_length')

    return input_data, targets, lr, keep_prob, summary_length, max_summary_length, text_length

In [30]:
def process_encoding_input(target_data, vocab_to_int, batch_size):
    '''Remove the last word id from each batch and concat the <GO> to the begining of each batch'''
    
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending], 1)

    return dec_input

In [31]:
def encoding_layer(rnn_size, sequence_length, num_layers, rnn_inputs, keep_prob):
    '''Create the encoding layer'''
    
    for layer in range(num_layers):
        with tf.variable_scope('encoder_{}'.format(layer)):
            cell_fw = tf.contrib.rnn.LSTMCell(rnn_size,
                                              initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_fw = tf.contrib.rnn.DropoutWrapper(cell_fw, 
                                                    input_keep_prob = keep_prob)

            cell_bw = tf.contrib.rnn.LSTMCell(rnn_size,
                                              initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_bw = tf.contrib.rnn.DropoutWrapper(cell_bw, 
                                                    input_keep_prob = keep_prob)

            enc_output, enc_state = tf.nn.bidirectional_dynamic_rnn(cell_fw, 
                                                                    cell_bw, 
                                                                    rnn_inputs,
                                                                    sequence_length,
                                                                    dtype=tf.float32)
    # Join outputs since we are using a bidirectional RNN
    enc_output = tf.concat(enc_output,2)
    
    return enc_output, enc_state

In [32]:
def training_decoding_layer(dec_embed_input, summary_length, dec_cell, initial_state, output_layer, 
                            vocab_size, max_summary_length):
    '''Create the training logits'''
    
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                        sequence_length=summary_length,
                                                        time_major=False)

    training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                       training_helper,
                                                       initial_state,
                                                       output_layer) 

    training_logits, _ = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                           output_time_major=False,
                                                           impute_finished=True,
                                                           maximum_iterations=max_summary_length)
    return training_logits

In [33]:
def inference_decoding_layer(embeddings, start_token, end_token, dec_cell, initial_state, output_layer,
                             max_summary_length, batch_size):
    '''Create the inference logits'''
    
    start_tokens = tf.tile(tf.constant([start_token], dtype=tf.int32), [batch_size], name='start_tokens')
    
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embeddings,
                                                                start_tokens,
                                                                end_token)
                
    inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                        inference_helper,
                                                        initial_state,
                                                        output_layer)
                
    inference_logits, _ = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                            output_time_major=False,
                                                            impute_finished=True,
                                                            maximum_iterations=max_summary_length)
    
    return inference_logits

In [34]:
# def decoding_layer(dec_embed_input, embeddings, enc_output, enc_state, vocab_size, text_length, summary_length, 
#                    max_summary_length, rnn_size, vocab_to_int, keep_prob, batch_size, num_layers):
def decoding_layer(dec_embed_input, embeddings, enc_output, enc_state, vocab_size, inputs_length, targets_length, 
                   max_target_length, rnn_size, vocab_to_int, keep_prob, batch_size, num_layers):
    '''Create the decoding cell and attention for the training and inference decoding layers'''
    
    for layer in range(num_layers):
        with tf.variable_scope('decoder_{}'.format(layer)):
            lstm = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            dec_cell = tf.contrib.rnn.DropoutWrapper(lstm, 
                                                     input_keep_prob = keep_prob)
    
    output_layer = Dense(vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    attn_mech = tf.contrib.seq2seq.BahdanauAttention(rnn_size,
                                                  enc_output,
                                                  text_length,
                                                  normalize=False,
                                                  name='BahdanauAttention')

    dec_cell = tf.contrib.seq2seq.AttentionWrapper(dec_cell,
                                                          attn_mech,
                                                          rnn_size)
            
#     initial_state = tf.contrib.seq2seq.DynamicAttentionWrapperState(enc_state[0],
#                                                                     _zero_state_tensors(rnn_size, 
#                                                                                         batch_size, 
#                                                                                         tf.float32)) 
    initial_state = dec_cell.zero_state(batch_size=batch_size,dtype=tf.float32).clone(cell_state=enc_state[0])
    with tf.variable_scope("decode"):
#         training_logits = training_decoding_layer(dec_embed_input, 
#                                                   summary_length, 
#                                                   dec_cell, 
#                                                   initial_state,
#                                                   output_layer,
#                                                   vocab_size, 
#                                                   max_summary_length)
        training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                            sequence_length=targets_length,
                                                            time_major=False)
        training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                           training_helper,
                                                           initial_state,
                                                           output_layer) 
        training_logits, _ ,_ = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                                  output_time_major=False,
                                                                  impute_finished=True,
                                                                  maximum_iterations=max_target_length)
        
    with tf.variable_scope("decode", reuse=True):
#         inference_logits = inference_decoding_layer(embeddings,  
#                                                     vocab_to_int['<GO>'], 
#                                                     vocab_to_int['<EOS>'],
#                                                     dec_cell, 
#                                                     initial_state, 
#                                                     output_layer,
#                                                     max_summary_length,
#                                                     batch_size)
        start_tokens = tf.tile(tf.constant([vocab_to_int['<GO>']], dtype=tf.int32), [batch_size], name='start_tokens')
        end_token = (tf.constant(vocab_to_int['<EOS>'], dtype=tf.int32))
        inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embeddings,
                                                                    start_tokens,
                                                                    end_token)
        inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                            inference_helper,
                                                            initial_state,
                                                            output_layer)
        inference_logits, _ ,_ = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                                   output_time_major=False,
                                                                   impute_finished=True,
                                                                   maximum_iterations=max_target_length)

    return training_logits, inference_logits

In [35]:
# def decoding_layer(dec_embed_input, embeddings, enc_output, enc_state, vocab_size, inputs_length, targets_length, 
#                    max_target_length, rnn_size, vocab_to_int, keep_prob, batch_size, num_layers, direction):
#     '''Create the decoding cell and attention for the training and inference decoding layers'''
    
#     with tf.name_scope("RNN_Decoder_Cell"):
#         for layer in range(num_layers):
#             with tf.variable_scope('decoder_{}'.format(layer)):
#                 lstm = tf.contrib.rnn.LSTMCell(rnn_size)
#                 dec_cell = tf.contrib.rnn.DropoutWrapper(lstm, 
#                                                          input_keep_prob = keep_prob)
    
#     output_layer = Dense(vocab_size,
#                          kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
#     attn_mech = tf.contrib.seq2seq.BahdanauAttention(rnn_size,
#                                                   enc_output,
#                                                   inputs_length,
#                                                   normalize=False,
#                                                   name='BahdanauAttention')
    
#     with tf.name_scope("Attention_Wrapper"):
#         dec_cell = tf.contrib.seq2seq.AttentionWrapper(dec_cell,
#                                                               attn_mech,
#                                                               rnn_size)
    
#     initial_state =  dec_cell.zero_state(batch_size=batch_size,dtype=tf.float32).clone(cell_state=enc_state)
# #     initial_state = tf.contrib.seq2seq.DynamicAttentionWrapperState(enc_state,
# #                                                                     _zero_state_tensors(rnn_size, 
# #                                                                                         batch_size, 
# #                                                                                         tf.float32))

#     with tf.variable_scope("decode"):
# #         training_logits = training_decoding_layer(dec_embed_input, 
# #                                                   targets_length, 
# #                                                   dec_cell, 
# #                                                   initial_state,
# #                                                   output_layer,
# #                                                   vocab_size, 
# #                                                   max_target_length)
#         training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
#                                                             sequence_length=targets_length,
#                                                             time_major=False)
#         training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
#                                                            training_helper,
#                                                            initial_state,
#                                                            output_layer) 
#         training_logits, _ ,_ = tf.contrib.seq2seq.dynamic_decode(training_decoder,
#                                             output_time_major=False,
#                                             impute_finished=True,
#                                             maximum_iterations=max_target_length)
        
#     with tf.variable_scope("decode", reuse=True):
# #         inference_logits = inference_decoding_layer(embeddings,  
# #                                                     vocab_to_int['<GO>'], 
# #                                                     vocab_to_int['<EOS>'],
# #                                                     dec_cell, 
# #                                                     initial_state, 
# #                                                     output_layer,
# #                                                     max_target_length,
# #                                                     batch_size)
#         start_tokens = tf.tile(tf.constant([vocab_to_int['<GO>']], dtype=tf.int32), [batch_size], name='start_tokens')
#         end_token = (tf.constant(vocab_to_int['<EOS>'], dtype=tf.int32))
#         inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embeddings,
#                                                                     start_tokens,
#                                                                     end_token)
#         inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
#                                                             inference_helper,
#                                                             initial_state,
#                                                             output_layer)
#         inference_logits, _ ,_ = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
#                                             output_time_major=False,
#                                             impute_finished=True,
#                                             maximum_iterations=max_target_length)

#     return training_logits, inference_logits

In [36]:
def seq2seq_model(input_data, target_data, keep_prob, text_length, summary_length, max_summary_length, 
                  vocab_size, rnn_size, num_layers, vocab_to_int, batch_size):
    '''Use the previous functions to create the training and inference logits'''
    
    # Use Numberbatch's embeddings and the newly created ones as our embeddings
    embeddings = word_embedding_matrix
    
    enc_embed_input = tf.nn.embedding_lookup(embeddings, input_data)
    enc_output, enc_state = encoding_layer(rnn_size, text_length, num_layers, enc_embed_input, keep_prob)
    
    dec_input = process_encoding_input(target_data, vocab_to_int, batch_size)
    dec_embed_input = tf.nn.embedding_lookup(embeddings, dec_input)
    
    training_logits, inference_logits  = decoding_layer(dec_embed_input, 
                                                        embeddings,
                                                        enc_output,
                                                        enc_state, 
                                                        vocab_size, 
                                                        text_length, 
                                                        summary_length, 
                                                        max_summary_length,
                                                        rnn_size, 
                                                        vocab_to_int, 
                                                        keep_prob, 
                                                        batch_size,
                                                        num_layers)
    
    return training_logits, inference_logits

In [37]:
def pad_sentence_batch(sentence_batch):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [vocab_to_int['<PAD>']] * (max_sentence - len(sentence)) for sentence in sentence_batch]

In [38]:
def get_batches(summaries, texts, batch_size):
    """Batch summaries, texts, and the lengths of their sentences together"""
    for batch_i in range(0, len(texts)//batch_size):
        start_i = batch_i * batch_size
        summaries_batch = summaries[start_i:start_i + batch_size]
        texts_batch = texts[start_i:start_i + batch_size]
        pad_summaries_batch = np.array(pad_sentence_batch(summaries_batch))
        pad_texts_batch = np.array(pad_sentence_batch(texts_batch))
        
        # Need the lengths for the _lengths parameters
        pad_summaries_lengths = []
        for summary in pad_summaries_batch:
            pad_summaries_lengths.append(len(summary))
        
        pad_texts_lengths = []
        for text in pad_texts_batch:
            pad_texts_lengths.append(len(text))
        
        yield pad_summaries_batch, pad_texts_batch, pad_summaries_lengths, pad_texts_lengths

In [39]:
# Set the Hyperparameters
epochs = 100
batch_size = 64
rnn_size = 256
num_layers = 2
learning_rate = 0.005
keep_probability = 0.75

In [40]:
# Build the graph
train_graph = tf.Graph()
# Set the graph to default to ensure that it is ready for training
with train_graph.as_default():
    
    # Load the model inputs    
    input_data, targets, lr, keep_prob, summary_length, max_summary_length, text_length = model_inputs()

    # Create the training and inference logits
    training_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                      targets, 
                                                      keep_prob,   
                                                      text_length,
                                                      summary_length,
                                                      max_summary_length,
                                                      len(vocab_to_int)+1,
                                                      rnn_size, 
                                                      num_layers, 
                                                      vocab_to_int,
                                                      batch_size)
    
    # Create tensors for the training logits and inference logits
    training_logits = tf.identity(training_logits.rnn_output, 'logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')
    
    # Create the weights for sequence_loss
    masks = tf.sequence_mask(summary_length, max_summary_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(learning_rate)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)
print("Graph is built.")

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Gr

## Training the Model

Since I am training this model on my MacBook Pro, it would take me days if I used the whole dataset. For this reason, I am only going to use a subset of the data, so that I can train it over night. Normally I use [FloydHub's](https://www.floydhub.com/) services for my GPU needs, but it would take quite a bit of time to upload the dataset and ConceptNet Numberbatch, so I'm not going to bother with that for this project.

I chose not use use the start of the subset because I didn't want to make it too easy for my model. The texts that I am using are closer to the median lengths; I thought this would be more fair.

In [41]:
# Subset the data for training
start = 200
end = start + 50000
sorted_summaries_short = sorted_summaries[start:end]
sorted_texts_short = sorted_texts[start:end]
print("The shortest text length:", len(sorted_summaries_short[0]))
print("The longest text length:",len(sorted_summaries_short[-1]))

The shortest text length: 7
The longest text length: 10


In [42]:
# Train the Model
learning_rate_decay = 0.95
min_learning_rate = 0.0005
display_step = 20 # Check training loss after every 20 batches
stop_early = 0 
stop = 100 # If the update loss does not decrease in 3 consecutive update checks, stop training
per_epoch = 3 # Make 3 update checks per epoch
update_check = (len(sorted_texts_short)//batch_size//per_epoch)-1

update_loss = 0 
batch_loss = 0
summary_update_loss = [] # Record the update losses for saving improvements in the model

checkpoint = "best_model.ckpt" 
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    
    # If we want to continue training a previous session
    #loader = tf.train.import_meta_graph("./" + checkpoint + '.meta')
    #loader.restore(sess, checkpoint)
    
    for epoch_i in range(1, epochs+1):
        update_loss = 0
        batch_loss = 0
        for batch_i, (summaries_batch, texts_batch, summaries_lengths, texts_lengths) in enumerate(
                get_batches(sorted_summaries_short, sorted_texts_short, batch_size)):
            start_time = time.time()
            _, loss = sess.run(
                [train_op, cost],
                {input_data: texts_batch,
                 targets: summaries_batch,
                 lr: learning_rate,
                 summary_length: summaries_lengths,
                 text_length: texts_lengths,
                 keep_prob: keep_probability})

            batch_loss += loss
            update_loss += loss
            end_time = time.time()
            batch_time = end_time - start_time

            if batch_i % display_step == 0 and batch_i > 0:
                print('Epoch {:>3}/{} Batch {:>4}/{} - Loss: {:>6.3f}, Seconds: {:>4.2f}'
                      .format(epoch_i,
                              epochs, 
                              batch_i, 
                              len(sorted_texts_short) // batch_size, 
                              batch_loss / display_step, 
                              batch_time*display_step))
                batch_loss = 0

            if batch_i % update_check == 0 and batch_i > 0:
                print("Average loss for this update:", round(update_loss/update_check,3))
                summary_update_loss.append(update_loss)
                
                # If the update loss is at a new minimum, save the model
                if update_loss <= min(summary_update_loss):
                    print('New Record!') 
                    stop_early = 0
                    saver = tf.train.Saver() 
                    saver.save(sess, checkpoint)

                else:
                    print("No Improvement.")
                    stop_early += 1
                    if stop_early == stop:
                        break
                update_loss = 0
            
                    
        # Reduce learning rate, but not below its minimum value
        learning_rate *= learning_rate_decay
        if learning_rate < min_learning_rate:
            learning_rate = min_learning_rate
        
        if stop_early == stop:
            print("Stopping Training.")
            break

Epoch   1/100 Batch   20/169 - Loss:  7.151, Seconds: 4.31
Epoch   1/100 Batch   40/169 - Loss:  5.143, Seconds: 4.64
Average loss for this update: 5.912
New Record!
Epoch   1/100 Batch   60/169 - Loss:  5.293, Seconds: 4.83
Epoch   1/100 Batch   80/169 - Loss:  5.200, Seconds: 5.00
Epoch   1/100 Batch  100/169 - Loss:  5.226, Seconds: 5.11
Average loss for this update: 5.241
New Record!
Epoch   1/100 Batch  120/169 - Loss:  5.240, Seconds: 5.30
Epoch   1/100 Batch  140/169 - Loss:  5.224, Seconds: 5.51
Epoch   1/100 Batch  160/169 - Loss:  5.157, Seconds: 5.60
Average loss for this update: 5.175
New Record!
Epoch   2/100 Batch   20/169 - Loss:  4.736, Seconds: 4.33
Epoch   2/100 Batch   40/169 - Loss:  4.544, Seconds: 4.63
Average loss for this update: 4.663
New Record!
Epoch   2/100 Batch   60/169 - Loss:  4.738, Seconds: 4.81
Epoch   2/100 Batch   80/169 - Loss:  4.686, Seconds: 5.03
Epoch   2/100 Batch  100/169 - Loss:  4.751, Seconds: 5.12
Average loss for this update: 4.746
No Im

Average loss for this update: 2.465
New Record!
Epoch  14/100 Batch   60/169 - Loss:  2.542, Seconds: 4.78
Epoch  14/100 Batch   80/169 - Loss:  2.462, Seconds: 4.94
Epoch  14/100 Batch  100/169 - Loss:  2.534, Seconds: 5.15
Average loss for this update: 2.514
No Improvement.
Epoch  14/100 Batch  120/169 - Loss:  2.547, Seconds: 5.31
Epoch  14/100 Batch  140/169 - Loss:  2.579, Seconds: 5.43
Epoch  14/100 Batch  160/169 - Loss:  2.566, Seconds: 5.57
Average loss for this update: 2.564
No Improvement.
Epoch  15/100 Batch   20/169 - Loss:  2.349, Seconds: 4.35
Epoch  15/100 Batch   40/169 - Loss:  2.305, Seconds: 4.60
Average loss for this update: 2.351
New Record!
Epoch  15/100 Batch   60/169 - Loss:  2.426, Seconds: 4.86
Epoch  15/100 Batch   80/169 - Loss:  2.327, Seconds: 4.98
Epoch  15/100 Batch  100/169 - Loss:  2.433, Seconds: 5.13
Average loss for this update: 2.4
No Improvement.
Epoch  15/100 Batch  120/169 - Loss:  2.444, Seconds: 5.36
Epoch  15/100 Batch  140/169 - Loss:  2.46

Epoch  27/100 Batch   80/169 - Loss:  1.528, Seconds: 4.95
Epoch  27/100 Batch  100/169 - Loss:  1.547, Seconds: 5.11
Average loss for this update: 1.552
No Improvement.
Epoch  27/100 Batch  120/169 - Loss:  1.544, Seconds: 5.24
Epoch  27/100 Batch  140/169 - Loss:  1.558, Seconds: 5.38
Epoch  27/100 Batch  160/169 - Loss:  1.535, Seconds: 5.57
Average loss for this update: 1.537
No Improvement.
Epoch  28/100 Batch   20/169 - Loss:  1.480, Seconds: 4.33
Epoch  28/100 Batch   40/169 - Loss:  1.448, Seconds: 4.56
Average loss for this update: 1.473
New Record!
Epoch  28/100 Batch   60/169 - Loss:  1.512, Seconds: 4.76
Epoch  28/100 Batch   80/169 - Loss:  1.481, Seconds: 4.89
Epoch  28/100 Batch  100/169 - Loss:  1.488, Seconds: 5.07
Average loss for this update: 1.499
No Improvement.
Epoch  28/100 Batch  120/169 - Loss:  1.492, Seconds: 5.25
Epoch  28/100 Batch  140/169 - Loss:  1.511, Seconds: 5.38
Epoch  28/100 Batch  160/169 - Loss:  1.483, Seconds: 5.50
Average loss for this update:

Average loss for this update: 1.087
No Improvement.
Epoch  40/100 Batch  120/169 - Loss:  1.069, Seconds: 5.25
Epoch  40/100 Batch  140/169 - Loss:  1.075, Seconds: 5.40
Epoch  40/100 Batch  160/169 - Loss:  1.083, Seconds: 5.54
Average loss for this update: 1.072
New Record!
Epoch  41/100 Batch   20/169 - Loss:  1.052, Seconds: 4.28
Epoch  41/100 Batch   40/169 - Loss:  1.022, Seconds: 4.58
Average loss for this update: 1.047
New Record!
Epoch  41/100 Batch   60/169 - Loss:  1.077, Seconds: 4.78
Epoch  41/100 Batch   80/169 - Loss:  1.058, Seconds: 4.96
Epoch  41/100 Batch  100/169 - Loss:  1.073, Seconds: 5.09
Average loss for this update: 1.064
No Improvement.
Epoch  41/100 Batch  120/169 - Loss:  1.058, Seconds: 5.28
Epoch  41/100 Batch  140/169 - Loss:  1.056, Seconds: 5.42
Epoch  41/100 Batch  160/169 - Loss:  1.057, Seconds: 5.54
Average loss for this update: 1.056
No Improvement.
Epoch  42/100 Batch   20/169 - Loss:  1.027, Seconds: 4.30
Epoch  42/100 Batch   40/169 - Loss:  1.

Epoch  53/100 Batch  140/169 - Loss:  0.855, Seconds: 5.42
Epoch  53/100 Batch  160/169 - Loss:  0.850, Seconds: 5.53
Average loss for this update: 0.845
No Improvement.
Epoch  54/100 Batch   20/169 - Loss:  0.824, Seconds: 4.27
Epoch  54/100 Batch   40/169 - Loss:  0.776, Seconds: 4.55
Average loss for this update: 0.809
New Record!
Epoch  54/100 Batch   60/169 - Loss:  0.839, Seconds: 4.75
Epoch  54/100 Batch   80/169 - Loss:  0.827, Seconds: 4.90
Epoch  54/100 Batch  100/169 - Loss:  0.835, Seconds: 5.07
Average loss for this update: 0.832
No Improvement.
Epoch  54/100 Batch  120/169 - Loss:  0.814, Seconds: 5.22
Epoch  54/100 Batch  140/169 - Loss:  0.837, Seconds: 5.51
Epoch  54/100 Batch  160/169 - Loss:  0.831, Seconds: 6.48
Average loss for this update: 0.827
No Improvement.
Epoch  55/100 Batch   20/169 - Loss:  0.797, Seconds: 4.34
Epoch  55/100 Batch   40/169 - Loss:  0.763, Seconds: 4.60
Average loss for this update: 0.788
New Record!
Epoch  55/100 Batch   60/169 - Loss:  0.

Average loss for this update: 0.679
No Improvement.
Epoch  67/100 Batch   20/169 - Loss:  0.656, Seconds: 4.29
Epoch  67/100 Batch   40/169 - Loss:  0.642, Seconds: 4.55
Average loss for this update: 0.652
New Record!
Epoch  67/100 Batch   60/169 - Loss:  0.670, Seconds: 4.75
Epoch  67/100 Batch   80/169 - Loss:  0.659, Seconds: 4.92
Epoch  67/100 Batch  100/169 - Loss:  0.678, Seconds: 5.09
Average loss for this update: 0.673
No Improvement.
Epoch  67/100 Batch  120/169 - Loss:  0.660, Seconds: 5.20
Epoch  67/100 Batch  140/169 - Loss:  0.668, Seconds: 5.38
Epoch  67/100 Batch  160/169 - Loss:  0.657, Seconds: 5.53
Average loss for this update: 0.659
No Improvement.
Epoch  68/100 Batch   20/169 - Loss:  0.681, Seconds: 4.29
Epoch  68/100 Batch   40/169 - Loss:  0.626, Seconds: 4.54
Average loss for this update: 0.654
No Improvement.
Epoch  68/100 Batch   60/169 - Loss:  0.665, Seconds: 4.77
Epoch  68/100 Batch   80/169 - Loss:  0.661, Seconds: 4.93
Epoch  68/100 Batch  100/169 - Loss:

Epoch  80/100 Batch   40/169 - Loss:  0.523, Seconds: 4.54
Average loss for this update: 0.556
New Record!
Epoch  80/100 Batch   60/169 - Loss:  0.572, Seconds: 5.48
Epoch  80/100 Batch   80/169 - Loss:  0.576, Seconds: 5.56
Epoch  80/100 Batch  100/169 - Loss:  0.552, Seconds: 5.13
Average loss for this update: 0.571
No Improvement.
Epoch  80/100 Batch  120/169 - Loss:  0.563, Seconds: 5.23
Epoch  80/100 Batch  140/169 - Loss:  0.555, Seconds: 5.42
Epoch  80/100 Batch  160/169 - Loss:  0.559, Seconds: 5.52
Average loss for this update: 0.555
New Record!
Epoch  81/100 Batch   20/169 - Loss:  0.565, Seconds: 4.30
Epoch  81/100 Batch   40/169 - Loss:  0.527, Seconds: 4.54
Average loss for this update: 0.551
New Record!
Epoch  81/100 Batch   60/169 - Loss:  0.563, Seconds: 4.76
Epoch  81/100 Batch   80/169 - Loss:  0.533, Seconds: 4.93
Epoch  81/100 Batch  100/169 - Loss:  0.557, Seconds: 5.12
Average loss for this update: 0.55
New Record!
Epoch  81/100 Batch  120/169 - Loss:  0.548, Seco

Epoch  93/100 Batch   60/169 - Loss:  0.510, Seconds: 4.78
Epoch  93/100 Batch   80/169 - Loss:  0.492, Seconds: 4.89
Epoch  93/100 Batch  100/169 - Loss:  0.493, Seconds: 5.07
Average loss for this update: 0.502
No Improvement.
Epoch  93/100 Batch  120/169 - Loss:  0.499, Seconds: 5.27
Epoch  93/100 Batch  140/169 - Loss:  0.484, Seconds: 5.38
Epoch  93/100 Batch  160/169 - Loss:  0.493, Seconds: 5.55
Average loss for this update: 0.485
New Record!
Epoch  94/100 Batch   20/169 - Loss:  0.496, Seconds: 4.28
Epoch  94/100 Batch   40/169 - Loss:  0.452, Seconds: 5.00
Average loss for this update: 0.473
New Record!
Epoch  94/100 Batch   60/169 - Loss:  0.481, Seconds: 4.83
Epoch  94/100 Batch   80/169 - Loss:  0.480, Seconds: 4.97
Epoch  94/100 Batch  100/169 - Loss:  0.501, Seconds: 5.19
Average loss for this update: 0.493
No Improvement.
Epoch  94/100 Batch  120/169 - Loss:  0.483, Seconds: 5.29
Epoch  94/100 Batch  140/169 - Loss:  0.477, Seconds: 5.44
Epoch  94/100 Batch  160/169 - Lo

## Making Our Own Summaries

To see the quality of the summaries that this model can generate, you can either create your own review, or use a review from the dataset. You can set the length of the summary to a fixed value, or use a random value like I have here.

In [43]:
def text_to_seq(text):
    '''Prepare the text for the model'''
    
    text = clean_text(text)
    return [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in text.split()]

In [49]:
# Create your own review or use one from the dataset
# input_sentence = "I hope that you found this project to be rather interesting and informative. One of my main recommendations for working with this dataset and model is either use a GPU, a subset of the dataset, or plenty of time to train your model. As you might be able to expect, the model will not be able to make good predictions just by seeing many reviews, it needs so see the reviews many times to be able to understand the relationship between words and between descriptions & summaries."
# text = text_to_seq(input_sentence)
random = np.random.randint(0,len(clean_texts))
input_sentence = clean_texts[random]
text = text_to_seq(clean_texts[random])

checkpoint = "./best_model.ckpt"

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    text_length = loaded_graph.get_tensor_by_name('text_length:0')
    summary_length = loaded_graph.get_tensor_by_name('summary_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')
    
    #Multiply by batch_size to match the model's input parameters
    answer_logits = sess.run(logits, {input_data: [text]*batch_size, 
                                      summary_length: [np.random.randint(5,8)], 
                                      text_length: [len(text)]*batch_size,
                                      keep_prob: 1.0})[0] 

# Remove the padding from the tweet
pad = vocab_to_int["<PAD>"] 

print('Original Text:', input_sentence)

print('\nText')
print('  Word Ids:    {}'.format([i for i in text]))
print('  Input Words: {}'.format(" ".join([int_to_vocab[i] for i in text])))

print('\nSummary')
print('  Word Ids:       {}'.format([i for i in answer_logits if i != pad]))
print('  Response Words: {}'.format(" ".join([int_to_vocab[i] for i in answer_logits if i != pad])))

INFO:tensorflow:Restoring parameters from ./best_model.ckpt
Original Text: optical properties 001 oriented nbp single crystals studied wide spectral range 6 mev 3 ev room temperature 10 k itinerant carriers lead drude like contribution optical response identify two pronounced phonon modes interband transitions starting already rather low frequencies comparing experimental findings calculated interband optical conductivity assign features observed measured conductivity certain interband transitions particular find transitions electronic bands spilt spin orbit coupling dominate interband conductivity nbp 100 mev low temperatures momentum relaxing scattering rate itinerant carriers nbp small leading macroscopic characteristic length scales momentum relaxation approximately 0 5 \mu

Text
  Word Ids:    [278, 479, 4786, 804, 4965, 463, 396, 4419, 2902, 203, 501, 126, 1874, 94, 9239, 2147, 266, 220, 986, 1847, 3137, 2961, 10113, 87, 2779, 278, 604, 11552, 43, 11638, 1034, 1184, 10875, 722, 5

Examples of reviews and summaries:
- Review(1): The coffee tasted great and was at such a good price! I highly recommend this to everyone!
- Summary(1): great coffee


- Review(2): This is the worst cheese that I have ever bought! I will never buy it again and I hope you won't either!
- Summary(2): omg gross gross


- Review(3): love individual oatmeal cups found years ago sam quit selling sound big lots quit selling found target expensive buy individually trilled get entire case time go anywhere need water microwave spoon know quaker flavor packets
- Summary(3): love it

## Summary

I hope that you found this project to be rather interesting and informative. One of my main recommendations for working with this dataset and model is either use a GPU, a subset of the dataset, or plenty of time to train your model. As you might be able to expect, the model will not be able to make good predictions just by seeing many reviews, it needs so see the reviews many times to be able to understand the relationship between words and between descriptions & summaries. 

In short, I'm pleased with how well this model performs. After creating numerous reviews and checking those from the dataset, I can happily say that most of the generated summaries are appropriate, some of them are great, and some of them make mistakes. I'll try to improve this model and if it gets better, I'll update my GitHub.

Thanks for reading!